In [1]:
import pandas as pd
import numpy as np

## Loading datas files and checking contents

In [2]:
airlines = pd.read_csv("data/airlines.csv")
airports = pd.read_csv("data/airports.csv")

In [3]:
airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [4]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


# Data Cleaning

## Fixing missing lat longs

In [5]:
airports[airports.LATITUDE.isnull()]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,NaN,NaN
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,NaN,NaN


In [6]:
# http://www.gcmap.com/airport/IATA_CODE
#IATA_CODE = ECP
airports.loc[96, 'LATITUDE'] = 30.358250
airports.loc[96, 'LONGITUDE'] = -85.795610
#IATA_CODE = PBG
airports.loc[234, 'LATITUDE'] = 44.650944
airports.loc[234, 'LONGITUDE'] = -73.468138
#IATA_CODE = UST
airports.loc[313, 'LATITUDE'] = 29.959250
airports.loc[313, 'LONGITUDE'] = -81.339721

In [7]:
# pre-set data types in certain columns to help reduce RAM usage
flights_dtype = {
'YEAR':                   np.uint16,
'MONTH':                  np.uint8,
'DAY':                    np.uint8,
'DAY_OF_WEEK':            np.uint8,
'AIRLINE':                np.object,
'FLIGHT_NUMBER':          np.uint16,
'TAIL_NUMBER':            np.object,
'ORIGIN_AIRPORT':         np.object,
'DESTINATION_AIRPORT':    np.object,
'SCHEDULED_DEPARTURE':    np.object,
'DEPARTURE_TIME':         np.object,
'WHEELS_OFF':             np.object,
'DISTANCE':               np.uint16,
'WHEELS_ON':              np.object,
'SCHEDULED_ARRIVAL':      np.object,
'ARRIVAL_TIME':           np.object,
'DIVERTED':               np.bool,
'CANCELLED':              np.int8,
'CANCELLATION_REASON':    np.object
}

In [8]:
flights = pd.read_csv("data/flights.csv",dtype=flights_dtype)

In [9]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,...,0408,-22.0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,...,0741,-9.0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,...,0811,5.0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,...,0756,-9.0,False,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,...,0259,-21.0,False,0,NaN,NaN,NaN,NaN,NaN,NaN


### Fixing Airport codes for the month of October

In [10]:
# October airport code are all 5 digits
flights[flights.MONTH==10][['MONTH','ORIGIN_AIRPORT','DESTINATION_AIRPORT']].head()

,MONTH,ORIGIN_AIRPORT,DESTINATION_AIRPORT
4385712,10,14747,11298
4385713,10,14771,13487
4385714,10,12889,13487
4385715,10,12892,13303
4385716,10,14771,11057


In [11]:
# Other month IATA codes are all IATA codes
flights[flights.MONTH==9][['MONTH','ORIGIN_AIRPORT','DESTINATION_AIRPORT']].head()

,MONTH,ORIGIN_AIRPORT,DESTINATION_AIRPORT
3920766,9,LAS,IAH
3920767,9,SFO,CLT
3920768,9,SFO,MSP
3920769,9,LAS,MSP
3920770,9,SFO,ORD


In [12]:
# Load mapping file to fix IATA_CODE for airports with 5 digital code in Oct
airport_mapping = pd.read_csv('data/airport_mapping.csv', index_col='IATA_CODE')

In [13]:
airport_mapping.head()

,ID,CITY,STATE,AIRPORT
IATA_CODE,,,,
01A,10001,Afognak Lake,AK,Afognak Lake Airport
03A,10003,Granite Mountain,AK,Bear Creek Mining Strip
04A,10004,Lik,AK,Lik Mining Camp
05A,10005,Little Squaw,AK,Little Squaw Airport
06A,10006,Kizhuyak,AK,Kizhuyak Bay


In [14]:
# Add a ID column in airport_df with the 5-digit airportID 
# based on airport_mapping_df
airports['ID'] = list(map((lambda x: airport_mapping.loc[x,'ID']), airports.IATA_CODE))

In [15]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,ID
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040,10135
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190,10136
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,10140
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183,10141
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447,10146


In [16]:
# Fix AIRPORT fields with AirportID(5 digits) instead of IATA_CODE
for item in airports.index:
    flights['ORIGIN_AIRPORT'].replace(str(airports.loc[item]['ID']), airports.loc[item]['IATA_CODE'], inplace=True)
    flights['DESTINATION_AIRPORT'].replace(str(airports.loc[item]['ID']), airports.loc[item]['IATA_CODE'], inplace=True)

In [17]:
# Issue fixed in Oct
flights[flights.MONTH==10][['MONTH','ORIGIN_AIRPORT','DESTINATION_AIRPORT']].head()

,MONTH,ORIGIN_AIRPORT,DESTINATION_AIRPORT
4385712,10,SEA,DFW
4385713,10,SFO,MSP
4385714,10,LAS,MSP
4385715,10,LAX,MIA
4385716,10,SFO,CLT


#### Label Cancellation Codes

In [18]:
cancel_reason = pd.DataFrame(
    [['NotCancel','Not Cancel'],
     ['A','Airline/Carrier'],
     ['B','Weather'],
     ['C','National Air System'],
     ['D','Security']], 
    columns=['Original_Code', 'Description'])

In [19]:
# Coding AIRPORT from IATA_CODE to index number
for item in cancel_reason.index:
    flights['CANCELLATION_REASON'].replace(
                    cancel_reason.loc[item]['Original_Code'], 
                    item, 
                    inplace=True)

In [20]:
#Set missing value to 0 to indicate 'Not Cancel'
flights['CANCELLATION_REASON'].fillna(value=0, inplace=True)
flights['CANCELLATION_REASON'] = flights['CANCELLATION_REASON'].astype('uint8')

In [21]:
# Change 32 to 16 bit, signed 16 bit should be able to represent the value ranges
flights_dtype_int16 = ['DEPARTURE_DELAY','ARRIVAL_DELAY']

In [22]:
for item in flights_dtype_int16:
    flights[item].fillna(value=0, inplace=True)
    flights[item] = flights[item].astype('int16')

In [23]:
flights_dtype_uint16 = ['TAXI_OUT','SCHEDULED_TIME','ELAPSED_TIME',
                       'AIR_TIME','TAXI_IN','AIR_SYSTEM_DELAY',
                       'SECURITY_DELAY','AIRLINE_DELAY',
                       'LATE_AIRCRAFT_DELAY','WEATHER_DELAY']

In [24]:
# Filling missing values with 0
for item in flights_dtype_uint16:
    flights[item].fillna(value=0, inplace=True)
    flights[item] = flights[item].astype('uint16')

### Code Military 4-digit Timestamps to 0-23 Hour Groups

In [25]:
time_col = ['SCHEDULED_DEPARTURE','DEPARTURE_TIME',
            'SCHEDULED_ARRIVAL','ARRIVAL_TIME']

In [26]:
for item in time_col:
    # Use -1 to represent no available timestamp for cancelled and diverted flights
    flights[item].fillna(value='-1', inplace=True)
    # Take only the first two digit as the hour group
    flights[item] = flights[item].str[0:2]
    #flights[item] = str(flights[item][0:2])
    # Map 2400 to 0, not exactly correct, but only a few data points
    #flights[item].replace(24, 0, inplace=True)

In [27]:
flights[['SCHEDULED_DEPARTURE','DEPARTURE_TIME',
           'SCHEDULED_ARRIVAL','ARRIVAL_TIME']].head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_ARRIVAL,ARRIVAL_TIME
0,00,23,04,04
1,00,00,07,07
2,00,00,08,08
3,00,00,08,07
4,00,00,03,02


In [28]:
flights = flights.drop(['DEPARTURE_TIME','WHEELS_OFF','WHEELS_ON','SCHEDULED_ARRIVAL','ARRIVAL_TIME','SCHEDULED_TIME',\
           'ELAPSED_TIME','AIR_TIME','TAXI_OUT','TAXI_IN','DIVERTED'], axis=1)

In [29]:
flights.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 20 columns):
YEAR                   5819079 non-null uint16
MONTH                  5819079 non-null uint8
DAY                    5819079 non-null uint8
DAY_OF_WEEK            5819079 non-null uint8
AIRLINE                5819079 non-null object
FLIGHT_NUMBER          5819079 non-null uint16
TAIL_NUMBER            5804358 non-null object
ORIGIN_AIRPORT         5819079 non-null object
DESTINATION_AIRPORT    5819079 non-null object
SCHEDULED_DEPARTURE    5819079 non-null object
DEPARTURE_DELAY        5819079 non-null int16
DISTANCE               5819079 non-null uint16
ARRIVAL_DELAY          5819079 non-null int16
CANCELLED              5819079 non-null int8
CANCELLATION_REASON    5819079 non-null uint8
AIR_SYSTEM_DELAY       5819079 non-null uint16
SECURITY_DELAY         5819079 non-null uint16
AIRLINE_DELAY          5819079 non-null uint16
LATE_AIRCRAFT_DELAY    5819079 non-null uint16
WE

In [30]:
#flights.to_pickle('cleaned_flight.pkl')

In [31]:
# load from pickle, this is much faster
#flights = pd.read_pickle('cleaned_flight.pkl')

In [32]:
#Convert to cleaned csv
flights.to_csv('cleaned_flight.csv')

## Identifying the airports in Texas, from where flights are originating

In [33]:
airports_tx = airports[airports['STATE']=='TX']

In [34]:
airports_tx

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,ID
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190,10136
6,ACT,Waco Regional Airport,Waco,TX,USA,31.61129,-97.23052,10155
16,AMA,Rick Husband Amarillo International Airport,Amarillo,TX,USA,35.21937,-101.70593,10279
22,AUS,Austin-Bergstrom International Airport,Austin,TX,USA,30.19453,-97.66987,10423
40,BPT,Jack Brooks Regional Airport (Southeast Texas ...,Beaumont/Port Arthur,TX,USA,29.95083,-94.02069,10728
44,BRO,Brownsville/South Padre Island International A...,Brownsville,TX,USA,25.90683,-97.42586,10747
65,CLL,Easterwood Airport,College Station,TX,USA,30.58859,-96.36382,11049
75,CRP,Corpus Christi International Airport,Corpus Christi,TX,USA,27.77036,-97.50122,11140
81,DAL,Dallas Love Field,Dallas,TX,USA,32.84711,-96.85177,11259
86,DFW,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720,11298


In [35]:
texas_airports = airports_tx["IATA_CODE"].values.tolist()

In [36]:
texas_airports

['ABI',
 'ACT',
 'AMA',
 'AUS',
 'BPT',
 'BRO',
 'CLL',
 'CRP',
 'DAL',
 'DFW',
 'ELP',
 'GGG',
 'GRK',
 'HOU',
 'HRL',
 'IAH',
 'LBB',
 'LRD',
 'MAF',
 'MFE',
 'SAT',
 'SJT',
 'SPS',
 'TYR']

In [37]:
texas_flights =  flights[flights['ORIGIN_AIRPORT'].isin(texas_airports)]

In [38]:
texas_flights.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687946 entries, 68 to 5819003
Data columns (total 20 columns):
YEAR                   687946 non-null uint16
MONTH                  687946 non-null uint8
DAY                    687946 non-null uint8
DAY_OF_WEEK            687946 non-null uint8
AIRLINE                687946 non-null object
FLIGHT_NUMBER          687946 non-null uint16
TAIL_NUMBER            686518 non-null object
ORIGIN_AIRPORT         687946 non-null object
DESTINATION_AIRPORT    687946 non-null object
SCHEDULED_DEPARTURE    687946 non-null object
DEPARTURE_DELAY        687946 non-null int16
DISTANCE               687946 non-null uint16
ARRIVAL_DELAY          687946 non-null int16
CANCELLED              687946 non-null int8
CANCELLATION_REASON    687946 non-null uint8
AIR_SYSTEM_DELAY       687946 non-null uint16
SECURITY_DELAY         687946 non-null uint16
AIRLINE_DELAY          687946 non-null uint16
LATE_AIRCRAFT_DELAY    687946 non-null uint16
WEATHER_DELAY        

### Get the csv file for Texas flights to process it for ML 

In [39]:
#Convert to cleaned csv
texas_flights.to_csv('texas_flights.csv')